# Data Query via PostGreSQL

## Inisialisasi Module

In [1]:
# General Package
import io
import logging
import warnings
import pandas as pd
import numpy as np
from copy import deepcopy

warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:,.2f}'.format
np.random.seed(1)

## Inisialization Function

In [2]:
from configparser import ConfigParser

confpt = '/mnt/d/WORKING/C01 PROJECTS/Configs.ini'
config = ConfigParser()

config.read(confpt)

['/mnt/d/WORKING/C01 PROJECTS/Configs.ini']

In [3]:
from psycopg2 import connect, DatabaseError
from psycopg2.extensions import STATUS_READY

# Database connection parameters
parameter = {"host"    : config.get('Postgre', 'HOST_IP'),
             "database": config.get('Postgre', 'DB_NAME'),
             "user"    : config.get('Postgre', 'DB_USER'),
             "password": config.get('Postgre', 'DB_PWD'),
            }
PostConnected = connect(**parameter)

In [4]:
def GetData(Koneksi   : object,
            query     : str,
            usepandas : bool = False,
           ) -> pd.DataFrame:
    assert len(query) > 10, 'Not sufficient query!'
    check = 'drop' in query[:10].lower() or \
            'insert' in query[:10].lower() or \
            'create' in query[:10].lower()
    if Koneksi.closed or Koneksi.status != STATUS_READY:
        print("Transaction is in an aborted state. Rolling back.")
        Koneksi.rollback()

    if not usepandas :
        postgr = Koneksi.cursor()
        postgr.execute(query)
        if not check:
            raw = postgr.fetchall()
            header = [item[0] for item in postgr.description]
            data = pd.DataFrame(raw, columns = header)
        postgr.close()

    else:
        data = pd.read_sql(query, Koneksi)

    Koneksi.commit()
    if not check:
        return data
    else:
        return 'Success'

In [5]:
from psycopg2 import DatabaseError
from psycopg2.extensions import STATUS_READY

def Pandas2Postgre(Data       : pd.DataFrame,
                   TableName  : str,
                   Connection : object,
                  ) -> bool:
    if Connection is None:
        print("Error: The database connection is not open.")
        return False

    values = list(Data.itertuples(index=False, name=None))
    Kolom = ', '.join([f'"{c}"' for c in Data.columns])
    try:
        with Connection.cursor() as postgr:
            if Connection.closed or Connection.status != STATUS_READY:
                print("Transaction is in an aborted state. Rolling back.")
                Connection.rollback()

            print(f"Dropping table '{TableName}' if it exists.")
            postgr.execute(f"DROP TABLE IF EXISTS \"{TableName}\"")

            print(f"Creating table '{TableName}'.")
            tipedata = {'int64'       : 'BIGINT',
                        'int32'       : 'INTEGER',
                        'float64'     : 'DOUBLE PRECISION',
                        'float32'     : 'REAL',
                        'object'      : 'TEXT',
                        'bool'        : 'BOOLEAN',
                        'datetime64[ns]' : 'TIMESTAMP',
                        'timedelta[ns]'  : 'INTERVAL',
                        'category'    : 'VARCHAR(255)',
                        'complex64'   : 'TEXT',
                        'complex128'  : 'TEXT',
                        'string'      : 'TEXT',
                       }
            
            Quote = list()
            for column, dtype in Data.dtypes.items():
                sql_type = tipedata.get(str(dtype), 'TEXT')
                Quote.append(f'"{column}" {sql_type}')

            Query = f"""CREATE TABLE "{TableName}" (
                        {', '.join(Quote)}
            );
            """
            postgr.execute(Query)

            print(f"Inserting data into '{TableName}'.")
            placeholders = ','.join(['%s'] * len(Data.columns))
            SQLdata = f"INSERT INTO \"{TableName}\"({Kolom}) VALUES({placeholders})"
            postgr.executemany(SQLdata, values)
        
        Connection.commit()
        print("Table created and data inserted successfully.")
        Success = True

    except DatabaseError as Arc:
        print(f"A database error occurred: {Arc}")
        Connection.rollback()
        Success = False

    finally:
        print('Transaction complete.')
        return Success

In [6]:
Query = '''
SELECT
	B."District_ID",
	A."stunting",
	A."gender",
	A."malnutrition_level",
	A."exclusive_milky",
	A."smoke_habit",
	A."purewater_access",
	A."healthy_toilet",
	A."difficult_acess"
FROM 
	"rawdata" AS A
LEFT JOIN
	"Label_District" AS B
ON
	A."district_name" = B."district_name"
ORDER BY
	"District_ID" ASC;
'''
Test = GetData(PostConnected, Query)
Test.sample(6)

,District_ID,stunting,gender,malnutrition_level,exclusive_milky,smoke_habit,purewater_access,healthy_toilet,difficult_acess
65,1,1,0,1,0,0,1,1,1
237,3,0,0,1,0,1,1,1,0
147,2,0,0,1,0,1,1,1,0
464,6,1,1,4,1,1,0,0,1
187,3,0,0,1,1,0,1,1,0
233,3,0,0,1,0,0,1,1,0
